
#            Calcul numérique du MGI d'un RRR par PNL avec contraintes #



|![alternative text](robot-rrr3-sol.png)|
|:--:| 
| *  Robot RRR  * |

## Objectif
On se propose de calculer de manière numérique un modèle géométrique inverse (MGI) d'un robot RRR en utilisant les fonctions de $scipy$ $optimize$ et en considérant des contraintes.

La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

La situation de l'outil est défini par le vecteur $X = (x, y, \theta)^t$ .


## Calcul du MGD
La fonction suivante calcule le modèle géométrique direct:


In [2]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = (x,y,z) 
def mgd(qdeg):
#### Paramètres du robot
    a,b,c=1,1,1
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
    x= a*c1*c2 + b*c1*c23
    y= a*s1*c2 + b*s1*c23
    z= a*s2 + b*s23 +c
    Xd=np.asarray([x,y,z])
    return Xd

In [3]:
#################################################   
# Calcul de J(q) du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: jacobienne(q) analytique:
def jacobienne(qdeg):
#### Paramètres du robot
    a,b=1,1
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
 
    Ja=np.array([[-a*s1*c2 -b*s1*c23, -a*c1*s2 -b*c1*s23,  -b*c1*s23], 
                [a*c1*c2 + b*c1*c23, -a*s1*s2 -b*s1*s23,  -b*s1*s23], 
                 [0, a*c2 + b*c23, b*c23]])

    return Ja

In [4]:
#################################################   
%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  # Fonction pour la 3D
from mpl_toolkits.mplot3d import proj3d
import numpy as np
import scipy
import time
from scipy.optimize import minimize
from scipy.optimize import root
import scipy.optimize as spop
from scipy import linalg


## 1 - Calcul du MGI sans contraintes

Trouver et utiliser les fonctions de $optimize$ permettant de calculer le MGI $(root, broyden, minimize,...)$.

Comparer les résultats avec votre implémentation


In [5]:
###################################################################################

def objectif(qdeg):
    return mgd(qdeg)-Xbut

def objectif_lineaire(qdeg):
    return np.linalg.norm(mgd(qdeg)-Xbut)

qinit= np.asarray([5., 0,  4.])

qbut= np.asarray([-45., 0,  45.])
## Calcul Xbut à partir de qbut
#Xbut= np.asarray(mgd(np.radians(qbut)))
Xbut= mgd(qbut)
print("Xbut=", Xbut[0], "Ybut = ", Xbut[1], "Z but = ", Xbut[2])
sol = root(objectif,qinit,jac = jacobienne,method = 'hybr')
print(sol.x)
print(mgd(sol.x))
print(sol.success)
solMinimize = minimize(objectif_lineaire,qinit)
print(solMinimize.x)
print(mgd(solMinimize.x))
print(solMinimize.success)
solbroyden = root(objectif,qinit,method = 'broyden1')
print(solbroyden.x%360)
print(mgd(solbroyden.x))
print(solbroyden.success)

Xbut= 1.2071067811865477 Ybut =  -1.2071067811865477 Z but =  1.7071067811865475
[-4.50000000e+01  1.44802271e-09  4.50000000e+01]
[ 1.20710678 -1.20710678  1.70710678]
True
[-4.50000000e+01  2.18455408e-09  4.50000000e+01]
[ 1.20710678 -1.20710678  1.70710678]
False
[134.99997762 180.00005839 315.00009215]
[ 1.20710762 -1.20710857  1.7071039 ]
True


## 2 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction $fun$ permettant de calculer la solution du MGI la plus proche de la configuration initiale $qinit$ donnée à la fonction $minimize$.

$res = minimize(fun, qinit, ....)$


In [6]:
def objectif_distance(qdeg):
    return np.linalg.norm(qdeg-qinit)


qbut= np.asarray([-45., 0,  45.])
# Calcul Xbut à partir de qbut
#Xbut= np.asarray(mgd(np.radians(qbut)))
Xbut= mgd(qbut)
print("Xbut=", Xbut[0], "Ybut = ", Xbut[1], "Z but = ", Xbut[2])

qinit= np.asarray([5., 0,  4.])
Xinit = mgd(qinit)
print("Xbut=", Xinit[0], "Ybut = ", Xinit[1], "Z but = ", Xinit[2])
constraints = ({'type': 'eq', 'fun': lambda qdeg: Xbut[0] - mgd(qdeg)[0]},
               {'type': 'eq', 'fun': lambda qdeg: Xbut[1] - mgd(qdeg)[1]},
               {'type': 'eq', 'fun': lambda qdeg:  Xbut[2] - mgd(qdeg)[2]})

solMinimize = minimize(objectif_distance,qinit,constraints=constraints,method = 'slsqp')
print(solMinimize.x)
print(mgd(solMinimize.x))
print(solMinimize.success)


SyntaxError: invalid syntax (1703499716.py, line 6)

## 3 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction permettant de calculer la solution du MGI en prenant en compte les butées du robot.

$q_1 \in [-120, 120]$

$q_2 \in [-45, 120]$

$q_3 \in [-120, 120]$


In [7]:
def objectif(qdeg):
    return mgd(qdeg)-Xbut

def objectif_lineaire(qdeg,Xbut):
    return np.linalg.norm(mgd(qdeg)-Xbut)

qinit= np.asarray([5., 0,  4.])

qbut= np.asarray([-45., 0,  45.])
## Calcul Xbut à partir de qbut
#Xbut= np.asarray(mgd(np.radians(qbut)))
Xbut= mgd(qbut)
print("Xbut=", Xbut[0], "Ybut = ", Xbut[1], "Z but = ", Xbut[2])

constraints = ({'type': 'ineq', 'fun': lambda qdeg: qdeg[0] - (-120)},
               {'type': 'ineq', 'fun': lambda qdeg: 120 - qdeg[0]},
               {'type': 'ineq', 'fun': lambda qdeg: qdeg[1] - (-45)},
               {'type': 'ineq', 'fun': lambda qdeg: 120 - qdeg[1]},
               {'type': 'ineq', 'fun': lambda qdeg: qdeg[2] - (-120)},
               {'type': 'ineq', 'fun': lambda qdeg: 120 - qdeg[2]})

solMinimize = minimize(objectif_lineaire,qinit,Xbut,constraints=constraints,method = 'slsqp')
print(solMinimize.x)
print(mgd(solMinimize.x))
print(solMinimize.success)

Xbut= 1.2071067811865477 Ybut =  -1.2071067811865477 Z but =  1.7071067811865475
[-4.50000107e+01  8.38132166e-05  4.49999130e+01]
[ 1.20710658 -1.20710703  1.7071082 ]
True
